In [151]:
# Import necessary packages and files
import mysql.connector
import requests
import config
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [287]:
from mysql.connector import errorcode
db_name = 'colleges'
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [285]:
# Connect to AWS
db_name = 'colleges'
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = "colleges"
)
cursor = cnx.cursor()

In [3]:
#import requests and beautiful soup in order to scrape
import requests
from bs4 import BeautifulSoup

In [4]:
#getting page with list of universities to then use to scrape other websites

page = requests.get("https://www.4icu.org/us/a-z/")
soup = BeautifulSoup(page.content, 'html.parser')


In [5]:
#soup_1 gets the areas of html where university name is stored
soup_1 = soup.find('tbody').find_all('a')

In [48]:
#getting university name from each line of html and adding it to list
#if not already on list. getting unique list of university names

list_ = [line.get_text() for line in soup_1]
list_of_colleges = []
unique_set = set()
for col_ in list_:
    if col_ not in unique_set:
        list_of_colleges.append(col_)
        unique_set.add(col_)
        

In [49]:
len(list_of_colleges)

1800

In [109]:
list_of_colleges

['Abilene Christian University',
 'Adams State University',
 'Adelphi University',
 'Adrian College',
 'Agnes Scott College',
 'Air Force Institute of Technology',
 'Alabama A&M University',
 'Alabama State University',
 'Alaska Bible College',
 'Alaska Pacific University',
 'Albany College of Pharmacy and Health Sciences',
 'Albany Law School',
 'Albany Medical College',
 'Albany State University',
 'Albertus Magnus College',
 'Albion College',
 'Albright College',
 'Alcorn State University',
 'Alderson Broaddus University',
 'Alfred State College',
 'Alfred University',
 'Alice Lloyd College',
 'Allegheny College',
 'Allen College',
 'Allen University',
 'Alliant International University',
 'Alma College',
 'Alvernia University',
 'Alverno College',
 'Amberton University',
 'American Baptist College',
 'American Film Institute Conservatory',
 'American International College',
 'American Jewish University',
 'American University',
 'Amherst College',
 'Anderson University',
 'Anderson

In [247]:
#this function checks for edge cases of non-usable colleges on list that don't return error-code
#takes in a list of colleges and identifies, for each college, if it does not return error,
# whether key identifier is present on page that signifies usability of page. 
# we found that 500 colleges on original list would run when queried, but did not return any information
#this function only adds colleges to new cleaned list that return relevant info.

def remove_bad_colleges(list_):
    list_of_colleges_1 = []

    for item in list_:
        formatted_college = format_college(item)
        url = f"https://www.collegedata.com/college/{formatted_college}"        
        page = requests.get(url)
        if page.status_code == 200:
            college_soup = BeautifulSoup(page.content, 'html.parser')
            a = college_soup.find('li', class_="breadcrumb-item active").get_text()
            if 'College Profile' in a:
                list_of_colleges_1.append(item)

    return list_of_colleges_1


In [248]:
list_of_colleges_cleaned = remove_bad_colleges(list_of_colleges)

In [250]:
len(list_of_colleges_cleaned)

1300

In [8]:
# this function takes in a string (college name) and reformats it for url query

def format_college(college):
    formatted_college = college.replace('&','').replace(' ','-')
    return formatted_college

In [118]:
# this function takes in a soup, and gets data on whether college is public or private:
# returns 1 for private, 0 for public

def get_private(soup):
    info = soup.find_all(class_='statbar__item')
    private = None
    for i in info:
        if 'Private' in i.get_text():
            private = 1
        elif 'Public'in i.get_text():
            private = 0
    return private

In [267]:
# this function takes in a soup, and gets data on undergrad size for a university:
# it checks for common input when data is missing in order to 
# be better able to turn data into appropriate type when data is present

def get_total_undergrad(soup):
    info = soup.find_all(class_='statbar__item')
    total_undergrad = None
    for i in info:
        sub_info = i.find_all('span')
        for i in sub_info:
            if 'Undergraduate' in i.get_text():
                if "Not reported" in sub_info[0].get_text():
                    total_undergrad = None
                else:
                    total_undergrad = int(sub_info[0].get_text().strip().replace(',',''))
    return total_undergrad

In [268]:
get_total_undergrad(soup_web)

15182

In [269]:
# this function takes in a soup, and gets data on admissions and incoming student profiles for a university:
# each block identifies the relevant area on the web page, loops through to check 
# for identifier of information, uses the index to loop through a different list of tags
# to get the data. Additionally, it checks for common input when data is missing in order to 
# be better able to turn data into appropriate type when data is present


def get_admissions_info(soup):

    info = soup.find_all(class_="dl-split-sm")[0]
    
    entrance_difficulty = None
    for i,s in enumerate(info.find_all('dt')):
        if 'Entrance' in s.get_text():
            entrance_difficulty = info.find_all('dd')[i].get_text().strip()
    
    admission = None
    for i,s in enumerate(info.find_all('dt')):
        if 'Rate' in s.get_text():
            if "Not reported" in info.find_all('dd')[i].get_text():
                admission_rate = None
            else:
                admission_rate = int(info.find_all('dd')[i].get_text().strip().split('%')[0])
    
    info_2 = soup.find_all(class_="dl-split-sm")[1]
    
    avg_gpa = None
    for i,s in enumerate(info_2.find_all('dt')):
        if 'GPA' in s.get_text():
            if "reported" in info.find_all('dd')[i].get_text():
                avg_gpa = None
            else:
                avg_gpa = info_2.find_all('dd')[i].get_text().strip()

    
    SAT_math = None
    for i,s in enumerate(info_2.find_all('dt')):
        if 'Math' in s.get_text():
            SAT_math = info_2.find_all('dd')[i].get_text().strip()
            if "Not reported" in SAT_math:
                SAT_math_avg = None
            else: 
                if 'average' in SAT_math:
                    SAT_math_avg = int(SAT_math.split()[0])
                else:
                    SAT_math_25p = int(SAT_math.split('-')[0])
                    SAT_math_75p = int(SAT_math.split('-')[1].split()[0])
                    SAT_math_avg = np.mean([SAT_math_25p,SAT_math_75p])

    SAT_EBRW = None
    for i,s in enumerate(info_2.find_all('dt')):
        if 'EBRW' in s.get_text():
            SAT_EBRW = info_2.find_all('dd')[i].get_text().strip()
            if "Not reported" in SAT_EBRW:
                SAT_EBRW_avg = None
            else:
                if 'average' in SAT_EBRW:
                    SAT_EBRW_avg = int(SAT_EBRW.split()[0])
                else:
                    SAT_EBRW_25p = int(SAT_EBRW.split('-')[0])
                    SAT_EBRW_75p = int(SAT_EBRW.split('-')[1].split()[0])
                    SAT_EBRW_avg = np.mean([SAT_EBRW_25p,SAT_EBRW_75p])
            

    return [entrance_difficulty, admission_rate, avg_gpa, SAT_math_avg, SAT_EBRW_avg]


In [271]:
# this function takes in a soup, and gets data on the cost of a university:
# each block identifies the relevant area on the web page, loops through to check 
# for identifier of information, uses the index to loop through a different list of tags
# to get the data. Additionally, it checks for common input when data is missing in order to 
# be better able to turn data into appropriate type when data is present

each loop locates the area in which the data is stored, checks to see if the relevant info is there
and then checks for common 

def get_cost_info(soup):
    info = soup.find_all(class_='card-body')[1]
    
    total_cost = None
    in_cost = None
    out_cost = None
    for i,s in enumerate(info.find_all('dt')):
        if 'Cost' in s.get_text():
            if 'state' in info.find_all('dd')[i].get_text():

                in_cost = int(info.find_all('dd')[i].get_text().split('$', 4)[1].split('O')[0].replace(',',''))
                out_cost = int(info.find_all('dd')[i].get_text().split('$', 4)[2].replace(',',''))

            else:
                if 'Not reported' in info.find_all('dd')[i].get_text() or 'Not available' in info.find_all('dd')[i].get_text():
                    total_cost = None
                else:
                    total_cost = int(info.find_all('dd')[i].get_text().strip().replace(',','').replace('$',''))

    tuition = None
    in_tuition = None
    out_tuition = None
    for i,s in enumerate(info.find_all('dt')):
        if 'Tuition' in s.get_text():
            if 'state' in info.find_all('dd')[i].get_text():
                in_tuition = int(info.find_all('dd')[i].get_text().split('$', 4)[1].split('O')[0].replace(',',''))
                out_tuition = int(info.find_all('dd')[i].get_text().split('$', 4)[2].replace(',',''))

            else:
                if 'Not reported' in info.find_all('dd')[i].get_text():
                    tuition = None
                else:
                    tuition = int(info.find_all('dd')[i].get_text().strip().replace(',','').replace('$',''))

    room_and_board = None
    for i,s in enumerate(info.find_all('dt')):
        if 'Room' in s.get_text():
            if 'Not reported' in info.find_all('dd')[i].get_text():
                room_and_board = None
            else:
                room_and_board = int(info.find_all('dd')[i].get_text().strip().replace(',','').replace('$',''))
            
    avg_pcent_need_met = None
    for i,s in enumerate(info.find_all('dt')):
        if 'Need' in s.get_text():
            if 'Not reported' in info.find_all('dd')[i].get_text():
                avg_pcent_need_met = None
            else:
                avg_pcent_need_met = int(info.find_all('dd')[i].get_text().strip().replace('%',''))
            
    avg_f_award = None
    for i,s in enumerate(info.find_all('dt')):
        if 'Freshman' in s.get_text():
            if 'Not reported' in info.find_all('dd')[i].get_text():
                avg_f_award = None
            else:
                avg_f_award = int(info.find_all('dd')[i].get_text().strip().replace(',','').replace('$',''))
            
    avg_indebted = None        
    for i,s in enumerate(info.find_all('dt')):
        if 'Indebtedness' in s.get_text():
            if 'Not reported' in info.find_all('dd')[i].get_text():
                avg_indebted = None
            else:
                avg_indebted = int(info.find_all('dd')[i].get_text().strip().replace(',','').replace('$',''))
    
    return [total_cost,in_cost,out_cost,tuition,in_tuition,out_tuition,room_and_board,avg_pcent_need_met,avg_f_award,avg_indebted]


In [273]:
# this function takes in a college and the corresponding soup page, 
# it calls on subfunctions that return relavent data from different sections of page
# then returns a flattened tuple from a list of lists

def get_info(college, college_soup):
    
    
    list_of_lists=[[college], [get_private(college_soup)], [get_total_undergrad(college_soup)],
           get_admissions_info(college_soup),get_cost_info(college_soup)]

    flattened = [val for sublist in list_of_lists for val in sublist]
    return tuple(flattened)


In [14]:
from tqdm import tnrange, tqdm_notebook as tqdm
from time import sleep

In [275]:
#this function 1) goes through list of college names,
# 2) reformats name in order to query in url
# 3) gets the page for each college and runs sub-function
#   that returns tuple with data for each function

def scrape(colleges_):
    list_of_tuples = []
    college_tuple = None
    
    with tqdm(total=len(colleges_)) as pbar:


        #iterate through titles    
        for college in colleges_:
            pbar.update(1)
            #add query title string to end of url
            college = college
            formatted_college = format_college(college)
            url = f"https://www.collegedata.com/college/{formatted_college}"        
            page = requests.get(url)
            if page.status_code == 200:
                college_soup = BeautifulSoup(page.content, 'html.parser')
                college_tuple = get_info(college, college_soup)  
            
            list_of_tuples.append(college_tuple) 

            sleep(0.001)
        
    return list_of_tuples

In [276]:
data = scrape(list_of_colleges_cleaned)

In [277]:
data

[('Abilene Christian University',
  1,
  3670,
  'Moderately difficult',
  51,
  '3.62',
  557.5,
  566,
  49792,
  None,
  None,
  34850,
  None,
  None,
  10350,
  71,
  26051,
  None),
 ('Adams State University',
  0,
  1991,
  'Moderately difficult',
  99,
  '3.18',
  480.0,
  495,
  None,
  22893,
  33909,
  None,
  9440,
  20456,
  8782,
  70,
  14726,
  22822),
 ('Adelphi University',
  1,
  5391,
  'Moderately difficult',
  74,
  '3.52',
  590.0,
  586,
  58710,
  None,
  None,
  38660,
  None,
  None,
  16030,
  44,
  22900,
  34980),
 ('Adrian College',
  1,
  1647,
  'Moderately difficult',
  56,
  '3.23',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  77,
  27013,
  27741),
 ('Agnes Scott College',
  1,
  996,
  'Moderately difficult',
  70,
  '3.76',
  580.0,
  627,
  57505,
  None,
  None,
  42690,
  None,
  None,
  12670,
  84,
  37669,
  30850),
 ('Alabama State University',
  0,
  3903,
  'Minimally difficult',
  98,
  '3.05',
  465.0,
  480,

In [293]:
#Table to insert into database
TABLES = {}
TABLES['college_info'] = (
    """CREATE TABLE college_info (
       college varchar(50),
       private varchar(5) ,
       total_undergrad varchar(50),
       entrance_difficulty varchar(20),
       admission_rate int(10),
       avg_gpa int(10), 
       SAT_math_avg int(10),
       SAT_EBRW_avg int(10),
       total_cost varchar(20),
       in_cost int(10), 
       out_cost int(10), 
       tuition varchar(20), 
       in_tuition int(10), 
       out_tuition int(10),
       room_and_board varchar(20), 
       avg_pcent_need_met varchar(20), 
       avg_f_award varchar(20), 
       avg_indebted varchar(20),
       PRIMARY KEY (college)
     ) ENGINE=InnoDB""")

In [294]:
# For loop to create each table 
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


cursor.close()


Creating table college_info: OK


True

In [295]:
cursor = cnx.cursor()
# Write a function to take parsed data and insert it into the DB
def insert_data(insert_statement, data):
    cursor.executemany(insert_statement, data)
    cnx.commit()

In [296]:
insert_statement = """INSERT INTO college_info (college, private, total_undergrad, entrance_difficulty, 
admission_rate, avg_gpa, SAT_math_avg, SAT_EBRW_avg,
total_cost, in_cost, out_cost, 
tuition, in_tuition, out_tuition,
room_and_board, avg_pcent_need_met, 
avg_f_award, avg_indebted
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [ ]:
len(unique_data)

In [297]:
#below function takes in the insert statement and list of tuples and inputs data into sql
insert_data(insert_statement, data)

In [278]:
pd.DataFrame.from_records(data).to_csv('sasha_backup_1.csv')